In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
from haystack.document_stores import InMemoryDocumentStore
from haystack.nodes import BM25Retriever, PromptModel, PromptNode, PromptTemplate, AnswerParser
from haystack.pipelines import Pipeline

from datasets import load_dataset

/home/catsmile/miniconda3/envs/py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-08-06 23:52:05.961760: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-06 23:52:07.038982: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-08-06 23:52:07.039286: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_

In [ ]:
# A DocumentStore stores the Documents that the question answering system uses to find answers to your questions.
document_store = InMemoryDocumentStore(use_bm25=True)

# Fetch dataset and write it to document_store
dataset = load_dataset("bilgeyucel/seven-wonders", split="train")
document_store.write_documents(dataset)

# initialize a BM25Retriever and make it use the InMemoryDocumentStore
retriever = BM25Retriever(document_store=document_store, top_k=2)

Updating BM25 representation...: 100%|██████████| 151/151 [00:00<00:00, 13832.01 docs/s]


In [ ]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
prompt_open_ai = PromptModel(model_name_or_path="text-davinci-003", api_key=OPENAI_API_KEY)

rag_prompt = PromptTemplate(
    prompt="""Synthesize a comprehensive answer from the following text for the given question.
                             Provide a clear and concise response that summarizes the key points and information presented in the text.
                             Your answer should be in your own words and be no longer than 50 words.
                             \n\n Related text: {join(documents)} \n\n Question: {query} \n\n Answer:""",
    # output_parser=AnswerParser(),
)

# prompt_node = PromptNode(model_name_or_path="google/flan-t5-large", default_prompt_template=rag_prompt)

prompt_node = PromptNode(prompt_open_ai, default_prompt_template=rag_prompt)

In [ ]:
pipe = Pipeline()
pipe.add_node(component=retriever, name="retriever", inputs=["Query"])
pipe.add_node(component=prompt_node, name="prompt_node", inputs=["retriever"])

In [ ]:
examples = [
    "Where is Gardens of Babylon?",
    "Why did people build Great Pyramid of Giza?",
    "What does Rhodes Statue look like?",
    "Why did people visit the Temple of Artemis?",
    "What is the importance of Colossus of Rhodes?",
    "What happened to the Tomb of Mausolus?",
    "How did Colossus of Rhodes collapse?",
]

output = pipe.run(query="What does Rhodes Statue look like?")

In [ ]:
output

{'results': ["Scholars don't know exactly what the statue looked like, but they do have a good idea of what the head and face looked like. It is believed to have had curly hair with evenly spaced spikes of bronze or silver flame radiating, similar to images found on contemporary Rhodian coins."],
 'invocation_context': {'query': 'What does Rhodes Statue look like?',
  'documents': [<Document: {'content': "Silver tetradrachm of Rhodes showing Helios and a rose (205–190 BC, 13.48 g)\nWhile scholars do not know what the statue looked like, they do have a good idea of what the head and face looked like, as it was of a standard rendering at the time. The head would have had curly hair with evenly spaced spikes of bronze or silver flame radiating, similar to the images found on contemporary Rhodian coins.[29]\n\nPossible locations[edit]\nThe old harbour entrance from inner embankment. The Fortress of St Nicholas is on right\nWhile scholars generally agree that anecdotal depictions of the Col